In [189]:
import pandas as pan
import matplotlib.pyplot as pd
import numpy as np
import seaborn as sns
pan.options.display.mpl_style = 'default'
%matplotlib inline 

/home/thakursc1/anaconda3/lib/python3.4/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/thakursc1/anaconda3/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [190]:
df = pan.read_csv('final.csv')


In [191]:
data  = df[['funded_amnt','loan_amnt','term','int_rate','installment','grade',
            'emp_length','home_ownership','verification_status','issue_d','purpose',
            'addr_state','dti','delinq_2yrs','open_acc','total_pymnt','revol_bal',
            'collections_12_mths_ex_med','application_type', 'acc_now_delinq']]

In [192]:
data['emp_length'].replace({'n/a':'10+ years'},inplace=True)

/home/thakursc1/anaconda3/lib/python3.4/site-packages/pandas/core/generic.py:2859: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [193]:
data.emp_length.value_counts()

10+ years    329227
2 years       77333
< 1 year      68832
3 years       68588
1 year        55863
5 years       54562
4 years       51478
7 years       43937
8 years       43039
6 years       42199
9 years       33983
dtype: int64

In [194]:
statuses = ['Fully Paid','Charged Off','Does not meet the credit policy. Status:Fully Paid',
            'Default','Does not meet the credit policy. Status:Charged Off','Late (31-120 days)']
df = df.loc[df['loan_status'].isin(statuses)]
data = data.iloc[df.index]

In [195]:
data.shape

(267470, 20)

In [196]:
#Change value of statuses to default or Non-default
categories  = ['Fully Paid','Does not meet the credit policy. Status:Fully Paid']
for cat in categories:
    df.loan_status.replace({cat:'normal'},inplace = True)
categories  = ['Charged Off','Default','Does not meet the credit policy. Status:Charged Off',
               'Late (31-120 days)']
for cat in categories:
    df.loan_status.replace({cat:'default'},inplace = True)

In [197]:
state = ['MN',
 'MO',
 'NV',
 'CT',
 'OR',
 'AL',
 'WI',
 'LA',
 'IN',
 'SC',
 'TN',
 'KY',
 'OK',
 'UT',
 'KS',
 'AR',
 'HI',
 'NM',
 'WV',
 'NH',
 'RI',
 'DC',
 'MT',
 'DE',
 'AK',
 'WY',
 'SD',
 'MS',
 'VT',
 'NE',
 'IA',
 'ME',
 'ND',
 'ID']

for cat in state:
    data.addr_state.replace({cat:'other'},inplace = True)
    
pur = ['home_improvement',
 'other',
 'major_purchase',
 'small_business',
 'car',
 'medical',
 'moving',
 'wedding',
 'house',
 'vacation',
 'educational',
 'renewable_energy']

for cat in pur:
    data.purpose.replace({cat:'other'},inplace = True)

owner = ['OWN','OTHER','NONE','ANY'] 
for cat in owner:
    data.home_ownership.replace({cat:'other'},inplace = True)

cat_grade = ['E','F','G']
for cat in cat_grade:
    data.grade.replace({cat:'E'},inplace = True)

In [198]:
data.issue_d = pan.to_datetime(data.issue_d)

In [199]:
data['issue_d_month'] = data.issue_d.dt.month
data['issue_d_quarter'] = data.issue_d.dt.quarter
data['issue_d_year'] = data.issue_d.dt.year

In [200]:
data.issue_d_year.value_counts(sort=False)/data.shape[0]*100

2007     0.170860
2008     0.889072
2009     1.962837
2010     4.662205
2011     7.358956
2012    18.584140
2013    27.311474
2014    27.648334
2015    11.412121
dtype: float64

In [201]:
data['differ'] = data['loan_amnt'] - data['funded_amnt']
data.drop('application_type',axis = 1,inplace=True)

In [202]:
data = data.sort('issue_d_year')
data = data.reset_index()

In [203]:
#1 train 
#0 test
data['type'] = 0
data[data.issue_d_year<=2013] = 1
test_train = data['type']
data.drop('type',axis = 1,inplace=True)

In [204]:
data.shape

(267470, 24)

In [205]:
columns_convert = list(data.dtypes[data.dtypes == 'float64'].index)
columns_convert
columns_convert1 = list(data.dtypes[(data.dtypes == 'object')|(data.dtypes == 'int64')].index)
columns_convert1

['index',
 'term',
 'grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'addr_state',
 'issue_d_month',
 'issue_d_quarter',
 'issue_d_year']

In [206]:
"""from sklearn.preprocessing import MinMaxScaler
def min_max_scale(df,columns_convert):
    min_max=MinMaxScaler()
    for i in columns_convert:
        df[i] = min_max.fit_transform(df[i])
        
    return df
    data = min_max_scale(data,columns_convert)"""

from sklearn.preprocessing import scale
def scaled(df,columns_convert):
    for i in columns_convert:
        df[i] = scale(df[i])
        
    return df

data = scaled(data,columns_convert)

In [207]:
data = pan.get_dummies(data,prefix=columns_convert1,columns=columns_convert1)

MemoryError: 

In [188]:
data.columns

Index(['index', 'funded_amnt', 'loan_amnt', 'term', 'int_rate', 'installment',
       'grade', 'emp_length', 'home_ownership', 'verification_status',
       'issue_d', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'open_acc',
       'total_pymnt', 'revol_bal', 'collections_12_mths_ex_med',
       'acc_now_delinq', 'issue_d_month', 'issue_d_quarter', 'issue_d_year',
       'differ'],
      dtype='object')

In [115]:
from sklearn.preprocessing import LabelEncoder
def lab_enc(df,columns_convert):
    le = LabelEncoder()
    for i in columns_convert:
        df[i] = le.fit_transform(df[i])
        
    return df

columns_convert = ['loan_status']
df = lab_enc(df,columns_convert)

In [116]:
df.loan_status.value_counts()

1    209409
0     58061
dtype: int64

In [118]:
data.drop('issue_d',axis=1,inplace = True)
data['loan_status'] = df.loan_status
data.dropna(axis=0,inplace=True)

ValueError: labels ['issue_d'] not contained in axis

In [125]:
#(267470, 49)
data.shape
t1 = data[test]

IndexError: indices are out-of-bounds

In [ ]:
print(data.loan_status.value_counts(),data.shape,train.shape,test.shape)

In [ ]:
train.to_csv('train_modelling.csv',index=False)
test.to_csv('test_modelling.csv',index=False)